## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,51.94,81,100,2.95,overcast clouds
1,1,Hermanus,ZA,-34.4187,19.2345,60.91,80,45,15.28,scattered clouds
2,2,Birjand,IR,32.8663,59.2211,30.67,37,0,0.00,clear sky
3,3,Lakatoro,VU,-16.0999,167.4164,79.54,72,99,4.97,overcast clouds
4,4,Cape Town,ZA,-33.9258,18.4232,62.11,66,0,12.66,clear sky


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 35
What is the maximum temperature you would like for your trip? 60


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,51.94,81,100,2.95,overcast clouds
5,5,Bredasdorp,ZA,-34.5322,20.0403,57.27,77,94,12.86,light rain
8,8,Dunedin,NZ,-45.8742,170.5036,58.35,78,100,4.63,moderate rain
9,9,Erzin,TR,36.9559,36.2000,54.32,85,100,1.63,light rain
10,10,Jamestown,US,42.0970,-79.2353,35.19,88,90,4.00,overcast clouds
13,13,East London,ZA,-33.0153,27.9116,58.14,100,40,3.44,light rain
22,22,Mineral Wells,US,32.8085,-98.1128,55.33,35,1,17.27,clear sky
24,24,Saint George,US,37.1041,-113.5841,59.34,41,1,0.00,clear sky
32,32,Constitucion,CL,-35.3333,-72.4167,51.67,92,45,1.34,scattered clouds
33,33,Penzance,GB,50.1186,-5.5371,51.76,75,100,4.38,overcast clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                187
City                   187
Country                185
Lat                    187
Lng                    187
Max Temp               187
Humidity               187
Cloudiness             187
Wind Speed             187
Current Description    187
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,51.94,overcast clouds,-46.6000,168.3333,
5,Bredasdorp,ZA,57.27,light rain,-34.5322,20.0403,
8,Dunedin,NZ,58.35,moderate rain,-45.8742,170.5036,
9,Erzin,TR,54.32,light rain,36.9559,36.2000,
10,Jamestown,US,35.19,overcast clouds,42.0970,-79.2353,
13,East London,ZA,58.14,light rain,-33.0153,27.9116,
22,Mineral Wells,US,55.33,clear sky,32.8085,-98.1128,
24,Saint George,US,59.34,clear sky,37.1041,-113.5841,
32,Constitucion,CL,51.67,scattered clouds,-35.3333,-72.4167,
33,Penzance,GB,51.76,overcast clouds,50.1186,-5.5371,


In [39]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,51.94,overcast clouds,-46.6000,168.3333,
5,Bredasdorp,ZA,57.27,light rain,-34.5322,20.0403,
8,Dunedin,NZ,58.35,moderate rain,-45.8742,170.5036,
9,Erzin,TR,54.32,light rain,36.9559,36.2000,
10,Jamestown,US,35.19,overcast clouds,42.0970,-79.2353,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna(subset = ["Hotel Name"], inplace=False)

In [47]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,51.94,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
5,Bredasdorp,ZA,57.27,light rain,-34.5322,20.0403,Bredasdorp Country Manor
8,Dunedin,NZ,58.35,moderate rain,-45.8742,170.5036,Scenic Hotel Southern Cross
9,Erzin,TR,54.32,light rain,36.9559,36.2000,Artemis Otel
10,Jamestown,US,35.19,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City name</dt><dd>{City}</dd>
<dt>Country code </dt><dd>{Country}</dd>
<dt>weather description</dt><dd>{Current Description}</dd>
<dt>Maximum temperature </dt><dd>{Max Temp}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))